In [722]:
import pandas as pd
import geopandas as gpd

In [723]:
# only keep cycleways

In [724]:
run = 7
agg = "segments"

In [725]:
if agg == "segments":
    # use segments
    gdf = gpd.read_file(f'/Users/alexandra/Documents/GitHub/road_network_classification/data/berlin_prio_vset/run{run}/berlin_segments_pred.shp', dtype={'ID': int}, crs=25833)
elif agg == "ways":
    # use ways
    gdf = gpd.read_file(f'/Users/alexandra/Documents/GitHub/road_network_classification/data/berlin_prio_vset/run{run}/berlin_ways_pred.shp', dtype={'ID': int}, crs=25833)

In [726]:
# use predefined road type as target
gdf['target_road_type'] = gdf['MAß_SEITE'].apply(lambda x: 'cycleway' if x == 'Beidseitig'
                                                 else ('road' if x in ('Gesamte Straße', 'Gesamte Straß') 
                                                       else None))

In [727]:
gdf['ROAD_TYPE_refined'] = gdf['ROAD_TYPE'].apply(lambda x: 'cycleway' if x in ('bike_lane', 'cycleway') else x)

In [728]:
len(gdf)

2906

In [729]:
gdf.target_road_type.value_counts()

target_road_type
cycleway    1029
road         674
Name: count, dtype: int64

In [730]:
# only keep ways where target_road_type and road_type match
gdf = gdf[(gdf['target_road_type'] == gdf['ROAD_TYPE_refined'])| (gdf['target_road_type'].isnull())].copy()

In [731]:
# how to choose bike lane or cycleway? first: which has more segments covered, second, which has more images

In [732]:
road_type_order = ['cycleway', 'road', 'pedestrian', 'other']

# select one road type per way
if agg == "ways":

    # for each road type, take the classification with the most votes (and segments if votes are tied)
    gdf.sort_values(['ID', 'SEGMENT_VO', 'AVG_IMG_CO'], ascending=False, inplace=True)
    gdf = gdf.groupby(['ID', 'ROAD_TYPE_refined']).first().reset_index()


    # Convert the ROAD_TYPE column to a categorical type with the specified order: use cycleways first
    gdf['ROAD_TYPE_refined'] = pd.Categorical(gdf['ROAD_TYPE_refined'], categories=road_type_order, ordered=True)
    gdf.sort_values(['ID', 'ROAD_TYPE_refined'], inplace=True)
    # convert back to string to save as gpkg
    gdf['ROAD_TYPE_refined'] = gdf['ROAD_TYPE_refined'].astype(str)
    gdf = gdf.groupby('ID').first().reset_index()

elif agg == "segments":
    gdf.sort_values(['ID', 'GROUP_NUM','SEGMENT_VO', 'AVG_IMG_CO'], ascending=False, inplace=True)
    gdf = gdf.groupby(['ID','GROUP_NUM', 'ROAD_TYPE_refined']).first().reset_index()

    # Convert the ROAD_TYPE column to a categorical type with the specified order: use cycleways first
    gdf['ROAD_TYPE_refined'] = pd.Categorical(gdf['ROAD_TYPE_refined'], categories=road_type_order, ordered=True)
    gdf.sort_values(['ID', 'GROUP_NUM', 'ROAD_TYPE_refined'], inplace=True)
    # convert back to string to save as gpkg
    gdf['ROAD_TYPE_refined'] = gdf['ROAD_TYPE_refined'].astype(str)
    gdf = gdf.groupby(['ID','GROUP_NUM']).first().reset_index()


In [733]:
gdf.crs = 25833

In [734]:
gdf['ID'] = gdf['ID'].astype(int)

In [735]:
if agg == "segments":
    file = "berlin_segments_pred_cycleways"
elif agg == "ways":
    file = "berlin_ways_pred_cycleways"

In [736]:
gdf.to_file(f'/Users/alexandra/Documents/GitHub/road_network_classification/data/berlin_prio_vset/run{run}/{file}.shp')

/var/folders/m7/jb7xk8h91p541g43zv2r8l740000gn/T/ipykernel_29464/2451296086.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(f'/Users/alexandra/Documents/GitHub/road_network_classification/data/berlin_prio_vset/run{run}/{file}.shp')
